In [1]:
from math import log
import QuantLib as ql
from collateralAgreement.collateralAgreement import CollateralAgreement
from instruments.interestRateInstrument.irs import IRS
from instruments.interestRateInstrument.swaption import Swaption
from sa_ccr.sa_ccr import SA_CCR
from utilities.Enums import SwapDirection
asdf=1


In [2]:
ca_sync = CollateralAgreement()
ca_sync.link_sa_ccr_instance(SA_CCR(ca_sync))
trade1 = IRS(notional=100, fixed_rate=0.00)
trade2 = Swaption(IRS(notional=1000, fixed_rate=0.00, swapDirection=SwapDirection.RECEIVER))
ca_sync.add_trades([trade1, trade2])

Let's try if our function is homogenious with a constant k.

We calculate our k as

\begin{align}
f(\alpha x) &= \alpha^kf(x) \\
\ln\left(\frac{f(\alpha x)}{f(x)}\right) / \ln(\alpha) &= k
\end{align}

with

* $f(x)$: SA CCR EAD
* $x$: Notional of the two swaps, i.e (100, 100)
* $\alpha$: 1.0001

In [3]:
alpha = 1.1
f=ca_sync.sa_ccr_model.get_ead()
alpha_trade1 = IRS(notional = 110, fixed_rate=0.00)
alpha_trade2 = Swaption(IRS(notional=1100, fixed_rate=0.00, swapDirection=SwapDirection.RECEIVER))
ca_sync.remove_all_trades()
ca_sync.add_trades([alpha_trade1, alpha_trade2])
f_alpha = ca_sync.sa_ccr_model.get_ead()
print(f_alpha)

k = log(f_alpha/f)/log(alpha)

0.3645260771618179


Now that we have a k let's try how well this works to approximate locally:

In [4]:
alpha_2 = 2
alpha_2_trade1 = IRS(notional = 200, fixed_rate=0.00)
alpha_2_trade2 = Swaption(IRS(notional=2000, fixed_rate=0.00, swapDirection=SwapDirection.RECEIVER))
ca_sync.remove_all_trades()
ca_sync.add_trades([alpha_2_trade1, alpha_2_trade2])
print('Calculated f_alpha2: %.10f ' %ca_sync.get_sa_ccr_model().get_ead())
print('Approximated f_alpha2: %.10f ' % (pow(alpha_2, k)*f))

Calculated f_alpha2: 0.6627746901 
Approximated f_alpha2: 0.6627746721 


With syncing the formula appears to be locally homogeneous. Let's try the same without syncing

In [5]:
ca_desync = CollateralAgreement()
ca_desync.link_sa_ccr_instance(SA_CCR(ca_desync))
trade1 = IRS(notional=100, fixed_rate=0.00)
trade2 = Swaption(IRS(notional=1000, fixed_rate=0.00, swapDirection=SwapDirection.RECEIVER))
ca_desync.add_trades([trade1, trade2])
ca_desync.sync_vm_model = False
ca_desync.sync_im_model = False

In [6]:
alpha = 1.1
f=ca_desync.sa_ccr_model.get_ead()
alpha_trade1 = IRS(notional = 110, fixed_rate=0.00)
alpha_trade2 = Swaption(IRS(notional=1100, fixed_rate=0.00, swapDirection=SwapDirection.RECEIVER))
ca_desync.remove_all_trades()
ca_desync.add_trades([alpha_trade1, alpha_trade2])
f_alpha = ca_desync.sa_ccr_model.get_ead()
print(f_alpha)

k = log(f_alpha/f)/log(alpha)

0.4496475527062012


Now that we have a k let's try how well this works to approximate locally:

In [7]:
alpha_2 = 2
alpha_2_trade1 = IRS(notional = 200, fixed_rate=0.00)
alpha_2_trade2 = Swaption(IRS(notional=2000, fixed_rate=0.00, swapDirection=SwapDirection.RECEIVER))
ca_desync.remove_all_trades()
ca_desync.add_trades([alpha_2_trade1, alpha_2_trade2])
print('Calculated f_alpha2: %.10f ' %ca_desync.get_sa_ccr_model().get_ead())
print('Approximated f_alpha2: %.10f ' % (pow(alpha_2, k)*f))

Calculated f_alpha2: 2.3804994374 
Approximated f_alpha2: 3.0493810580 


In [8]:
print(ca_desync.get_C())
print(ca_desync.sa_ccr_model.get_multiplier())
print(ca_sync.get_C())
print(ca_desync.sa_ccr_model.get_multiplier())

6.868626957525102
0.7050815295151583
13.737253887918328
0.7050815295151583


Finally, let's see how a model performs that has adjusted VM but no adjusted IM.

In [9]:
ca_only_vm = CollateralAgreement()
ca_only_vm.link_sa_ccr_instance(SA_CCR(ca_only_vm))
trade1 = IRS(notional=100, fixed_rate=0.00)
trade2 = Swaption(IRS(notional=1000, fixed_rate=0.00, swapDirection=SwapDirection.RECEIVER))
ca_only_vm.add_trades([trade1, trade2])
ca_only_vm.sync_im_model = False

In [10]:
alpha = 1.1
f=ca_only_vm.sa_ccr_model.get_ead()
alpha_trade1 = IRS(notional = 110, fixed_rate=0.00)
alpha_trade2 = Swaption(IRS(notional=1100, fixed_rate=0.00, swapDirection=SwapDirection.RECEIVER))
ca_only_vm.remove_all_trades()
ca_only_vm.add_trades([alpha_trade1, alpha_trade2])
f_alpha = ca_only_vm.sa_ccr_model.get_ead()
print(f_alpha)

k = log(f_alpha/f)/log(alpha)

0.41489175746196183


Now that we have a k let's try how well this works to approximate locally:

In [11]:
alpha_2 = 2
alpha_2_trade1 = IRS(notional = 200, fixed_rate=0.00)
alpha_2_trade2 = Swaption(IRS(notional=2000, fixed_rate=0.00, swapDirection=SwapDirection.RECEIVER))
ca_only_vm.remove_all_trades()
ca_only_vm.add_trades([alpha_2_trade1, alpha_2_trade2])
print('Calculated f_alpha2: %.10f ' %ca_only_vm.get_sa_ccr_model().get_ead())
print('Approximated f_alpha2: %.10f ' % (pow(alpha_2, k)*f))





Calculated f_alpha2: 1.4275154291 
Approximated f_alpha2: 1.6987435888 
